# Singerwise classifiers

In this notebook we carry out the SVM training and cross validation to predict from the kinematic contours whether a given note is stable or non-stable. This notebook contains the PVA-WE and PVA-E variants.

Note: This code will if and only if "Subseq dict" pickle files are ready (refer to notebook 2. Subseq Dict Generation, if they are not)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
import time
import pickle

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

In [4]:
for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:
# for singer in ['AG']:
    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])

    #PVA-LR
    drop_list = [x for x in data.columns.values if '3d' in x]
    data = data.drop(columns = drop_list)
    
#     display(data)
    
    gdata = data.copy()
#     display(gdata)
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(vc)
    print('Total:', (vc[0]+vc[1]))
    print('% Stable:', vc[1]/(vc[0]+vc[1]))

AG


1    956
0    822
Name: Target, dtype: int64

Total: 1778
% Stable: 0.5376827896512936
AK


0    1173
1     973
Name: Target, dtype: int64

Total: 2146
% Stable: 0.4534016775396086
AP


...

Total: 973
% Stable: 0.5282631038026722
SM


0    1261
1    1067
Name: Target, dtype: int64

Total: 2328
% Stable: 0.4583333333333333
SS


0    1419
1     645
Name: Target, dtype: int64

Total: 2064
% Stable: 0.3125


# Iterating Over Singers to Find SVM performance

## PVA-WE (All Wrist and Elbow contours included)

In [10]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:

    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500 
    # Read subseq dict file and reject notes outside [0.5,5]s if any
    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])
    
    data = []
    
    # Find mean and standard deviations of gesture contours
    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])
    # Drop unnecessary columns
    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])

    #PVA-WE: Only drop the '3d' columns, keep the rest
    drop_list = [x for x in data.columns.values if '3d' in x]
    data = data.drop(columns = drop_list)
    
    display(data)
    
    gdata = data.copy() # just making a copy to keep "data" unchanged
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(vc)
    # Training data (features) for cross val
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    
    # Training data (labels) for cross val
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])
    
    # Grid Search Parameters
    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    # Carry out cross validation and grid search, with 'refit' as f1-score
    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)

    # Analyze the cross validation results
    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)
    
    # Use best model to find predicted labels for each sample 
    # and calculate accuracy and f1 score
    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)

    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,ns
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,st
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,ns
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,-0.134559,0.122613,0.280415,-1.325317,2.835985,-0.260186,-5.081641,4.599241,0.014753,-0.460851,1.009332,-0.061194,0.636525,0.809240,0.205029,0.046481,0.096018,0.165208,0.144414,0.122563,0.176834,0.021546,0.015803,0.238436,0.101687,0.083646,1.037432,1.039591,1.054718,0.841924,3.636886,3.704609,0.932534,0.283905,0.550680,0.517793,0.527113,0.465152,5.793727,4.177815,5.752815,5.038683,10.870444,14.546414,5.263183,1.253753,2.975949,2.842216,st
4,1.348107,-0.702664,-0.242119,0.586842,1.487701,0.981908,2.241804,2.328519,0.546397,-0.670337,1.535956,-0.325972,0.049764,-0.167955,2.220086,0.579849,3.138152,1.563004,-1.213215,0.696194,0.869956,-0.017510,0.649386,0.972920,-0.014977,-0.386541,-6.077499,-7.593542,-9.219517,-6.426305,-0.664479,11.627751,-

1    956
0    822
Name: Target, dtype: int64

Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 12
Best parameters: {'svc__C': 1000, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.9376168965242361
AK


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,ns
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,ns
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,ns
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,1.807645,2.409392,-3.367885,0.039247,-0.124867,-3.633747,1.006198,1.728555,-2.118457,1.524946,-1.933401,-1.734313,0.055037,0.083757,0.090768,0.093103,0.088409,0.077677,0.007605,0.011022,0.074370,0.022312,0.056682,0.066486,0.287142,0.293162,0.557674,0.035797,0.038220,0.657216,0.167192,0.296570,0.593096,0.321005,0.339822,0.349130,0.496402,1.095084,4.150613,0.138224,0.259974,4.122407,0.336233,1.154491,1.983754,2.849388,1.361818,1.903463,st
4,0.573212,-0.429480,-0.772674,-0.232534,0.228691,-0.336812,0.729182,0.914745,-1.057484,0.262108,-0.531668,-1.451984,0.068485,0.156342,-0.734473,0.017660,-0.005047,-0.536769,0.014978,0.161038,-0.405075,-0.333299,-0.311330,-0.324999,-0.493145,1.562755,-0.202625,-0.008624,0.041849,-0.

...

0    1261
1    1067
Name: Target, dtype: int64

Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.8813545124961563
SS


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,ns
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,st
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,st
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,3.455696,4.482670,-0.273281,-4.555621,2.654418,-0.364214,-3.433900,0.164537,-0.431473,2.506595,1.208996,0.119725,0.330573,0.437058,0.149386,0.206251,0.203902,0.119565,0.467703,0.244265,0.106850,0.206814,0.071709,0.061714,1.659890,0.988000,0.270303,1.212122,0.846129,0.440764,1.261817,0.819535,0.790822,0.596523,0.923129,0.483451,7.347214,3.738326,1.615067,3.057386,4.260253,2.524017,3.311507,6.603217,4.808932,7.503954,4.460815,2.778396,ns
4,-1.552136,1.277841,1.075523,-0.505354,-0.573255,0.102790,-0.752685,1.293043,-0.363418,-0.554459,-0.450208,-1.156510,0.312514,0.226474,-0.268152,0.110754,-0.000756,-0.086211,0.227869,0.145903,0.263830,-0.178236,-0.333769,-0.229442,0.888212,4.329999,1.066489,0.587468,0.130098,1.691850,-1.277498,

0    1419
1     645
Name: Target, dtype: int64

Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.7727106594085108


In [2]:
def chance_F1(N0, N1):
    prec = N1/(N0+N1)
    rec = 1
    ch_f1 = 2*prec*rec/(prec+rec)
    return ch_f1

In [12]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.93307,0.93762,0.699342
1,AK,1173,973,0.91332,0.90516,0.623918
2,AP,1305,1180,0.90542,0.89827,0.643929
3,CC,1917,1048,0.89613,0.85385,0.522303
4,MG,821,1107,0.89990,0.91250,0.729489
5,MP,987,1265,0.88369,0.89568,0.719363
6,NM,1107,739,0.91711,0.89700,0.571760
7,RV,1039,681,0.88370,0.84953,0.567264
8,SCh,459,514,0.93319,0.93862,0.691325
9,SM,1261,1067,0.89045,0.88135,0.628571


# PVA-E (Only Elbow)

In [26]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

# Same code as PVA-WE except dropping some more columns (Wrist) before training

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:

    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])
    
    gdata = data.copy()
    
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(gdata)
    display(vc)
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    
    #PVA-E: Drop 3d columns, and Wrist columns
    drop_list = [x for x in X.columns.values if '3d' in x or 'Wrist' in x]
    X = X.drop(columns = drop_list)
    print('Columns included:', X.columns.values)
    
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])

    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)


    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)

    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)



    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid
    
#     display(results)

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,0.239863,1.698129,1.639468,2.452563,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,7.632676,7.650916,7.199229,12.047121,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,0.195151,0.854034,0.953142,1.188277,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,4.355223,2.669664,2.760062,5.381629,0
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,0.097496,1.194241,1.068156,2.996836,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,3.197070,6.036242,4.899836,14.006587,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.052214,0.666526,0.554291,2.758534,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,2.291594,3.981845,2.569355,11.180447,1
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.277212,3.086626,1.620276,3.760366,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,8.906563,15.135886,8.142713,26.922117,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,0.182422,0.991676,0.706405,2.022844,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,3.496218,3.272292,2.275699,11.566228,0
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,0.133230,1.382107,0.740968,3.661150,-0.134559,0.12

1    956
0    822
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'RElbow_x_m' 'RElbow_y_m'
 'RElbow_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'LElbow_accl_x_m'
 'LElbow_accl_y_m' 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'RElbow_x_s'
 'RElbow_y_s' 'RElbow_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'LElbow_accl_x_s' 'LElbow_accl_y_s' 'LElbow_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.9158632088461853
AK


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,2.074682,1.017248,1.856553,1.123659,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,10.267226,5.354898,8.993397,6.269162,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,1.321782,0.714755,0.978453,0.388700,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,4.316787,2.195364,4.123532,2.086301,0
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,1.677389,0.938211,1.530244,1.020210,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,8.903397,4.709948,7.516006,5.893906,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,1.353965,0.643009,1.038240,0.396043,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,4.489218,2.357535,4.469689,1.993289,0
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,1.482856,1.038457,1.370255,1.004904,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,8.184738,4.957916,6.578263,5.447635,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,1.155603,0.707483,0.932994,0.361364,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,4.165420,2.323531,3.945540,1.937088,0
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,0.676299,0.511897,0.804668,0.8

...

0    1261
1    1067
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'RElbow_x_m' 'RElbow_y_m'
 'RElbow_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'LElbow_accl_x_m'
 'LElbow_accl_y_m' 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'RElbow_x_s'
 'RElbow_y_s' 'RElbow_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'LElbow_accl_x_s' 'LElbow_accl_y_s' 'LElbow_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.8456691628701476
SS


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,2.672908,1.087595,3.003405,1.837792,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,6.693822,3.637398,11.719747,6.435866,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,1.212221,0.769910,0.799974,1.100834,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,3.072107,2.505105,4.543724,3.403451,0
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,2.159400,0.667067,6.369831,1.201926,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,5.682507,2.659807,26.248907,6.663064,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,0.720230,0.421421,5.176665,0.540927,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,2.393053,1.242323,13.407714,3.240611,1
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,6.808293,2.144596,4.187057,2.917348,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,7.301063,5.718990,28.685607,7.125793,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,0.601989,0.384581,0.496834,0.357974,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,2.699513,3.241842,29.361424,3.073313,1
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,1.636551,2.825120,1.280379,2.787136,3.

0    1419
1     645
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'RElbow_x_m' 'RElbow_y_m'
 'RElbow_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'LElbow_accl_x_m'
 'LElbow_accl_y_m' 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'RElbow_x_s'
 'RElbow_y_s' 'RElbow_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'LElbow_accl_x_s' 'LElbow_accl_y_s' 'LElbow_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.7119428228092938


In [28]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.90945,0.91587,0.699342
1,AK,1173,973,0.87793,0.86455,0.623918
2,AP,1305,1180,0.86276,0.85262,0.643929
3,CC,1917,1048,0.87251,0.82529,0.522303
4,MG,821,1107,0.84697,0.86830,0.729489
5,MP,987,1265,0.84903,0.86865,0.719363
6,NM,1107,739,0.87489,0.84470,0.571760
7,RV,1039,681,0.85116,0.80621,0.567264
8,SCh,459,514,0.88697,0.89738,0.691325
9,SM,1261,1067,0.85994,0.84567,0.628571


# VA-WE (We do not report this in paper)

In [15]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:

    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])
    
    gdata = data.copy()
    
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(gdata)
    display(vc)
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    
    #VA-WE
    drop_list = [x for x in X.columns.values if '3d' in x or ('vel' not in x and 'accl' not in x)]
    X = X.drop(columns = drop_list)
    print('Columns included:', X.columns.values)
    
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])

    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)


    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)

    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)



    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid
    
#     display(results)

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,0.239863,1.698129,1.639468,2.452563,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,7.632676,7.650916,7.199229,12.047121,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,0.195151,0.854034,0.953142,1.188277,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,4.355223,2.669664,2.760062,5.381629,0
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,0.097496,1.194241,1.068156,2.996836,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,3.197070,6.036242,4.899836,14.006587,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.052214,0.666526,0.554291,2.758534,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,2.291594,3.981845,2.569355,11.180447,1
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.277212,3.086626,1.620276,3.760366,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,8.906563,15.135886,8.142713,26.922117,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,0.182422,0.991676,0.706405,2.022844,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,3.496218,3.272292,2.275699,11.566228,0
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,0.133230,1.382107,0.740968,3.661150,-0.134559,0.12

1    956
0    822
Name: Target, dtype: int64

Columns included: ['RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m' 'LWrist_vel_x_m'
 'LWrist_vel_y_m' 'LWrist_vel_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m'
 'LElbow_vel_z_m' 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m'
 'RWrist_accl_x_m' 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m' 'RElbow_accl_z_m'
 'RWrist_vel_x_s' 'RWrist_vel_y_s' 'RWrist_vel_z_s' 'LWrist_vel_x_s'
 'LWrist_vel_y_s' 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'LWrist_accl_x_s'
 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s' 'LElbow_accl_y_s'
 'LElbow_accl_z_s' 'RElbow_accl_x_s' 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale

,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,2.074682,1.017248,1.856553,1.123659,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,10.267226,5.354898,8.993397,6.269162,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,1.321782,0.714755,0.978453,0.388700,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,4.316787,2.195364,4.123532,2.086301,0
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,1.677389,0.938211,1.530244,1.020210,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,8.903397,4.709948,7.516006,5.893906,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,1.353965,0.643009,1.038240,0.396043,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,4.489218,2.357535,4.469689,1.993289,0
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,1.482856,1.038457,1.370255,1.004904,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,8.184738,4.957916,6.578263,5.447635,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,1.155603,0.707483,0.932994,0.361364,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,4.165420,2.323531,3.945540,1.937088,0
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,0.676299,0.511897,0.804668,0.8

...

0    1261
1    1067
Name: Target, dtype: int64

Columns included: ['RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m' 'LWrist_vel_x_m'
 'LWrist_vel_y_m' 'LWrist_vel_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m'
 'LElbow_vel_z_m' 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m'
 'RWrist_accl_x_m' 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m' 'RElbow_accl_z_m'
 'RWrist_vel_x_s' 'RWrist_vel_y_s' 'RWrist_vel_z_s' 'LWrist_vel_x_s'
 'LWrist_vel_y_s' 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'LWrist_accl_x_s'
 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s' 'LElbow_accl_y_s'
 'LElbow_accl_z_s' 'RElbow_accl_x_s' 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale

,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,2.672908,1.087595,3.003405,1.837792,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,6.693822,3.637398,11.719747,6.435866,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,1.212221,0.769910,0.799974,1.100834,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,3.072107,2.505105,4.543724,3.403451,0
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,2.159400,0.667067,6.369831,1.201926,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,5.682507,2.659807,26.248907,6.663064,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,0.720230,0.421421,5.176665,0.540927,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,2.393053,1.242323,13.407714,3.240611,1
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,6.808293,2.144596,4.187057,2.917348,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,7.301063,5.718990,28.685607,7.125793,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,0.601989,0.384581,0.496834,0.357974,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,2.699513,3.241842,29.361424,3.073313,1
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,1.636551,2.825120,1.280379,2.787136,3.

0    1419
1     645
Name: Target, dtype: int64

Columns included: ['RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m' 'LWrist_vel_x_m'
 'LWrist_vel_y_m' 'LWrist_vel_z_m' 'LElbow_vel_x_m' 'LElbow_vel_y_m'
 'LElbow_vel_z_m' 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m'
 'RWrist_accl_x_m' 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m' 'RElbow_accl_z_m'
 'RWrist_vel_x_s' 'RWrist_vel_y_s' 'RWrist_vel_z_s' 'LWrist_vel_x_s'
 'LWrist_vel_y_s' 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s'
 'LElbow_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'LWrist_accl_x_s'
 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s' 'LElbow_accl_y_s'
 'LElbow_accl_z_s' 'RElbow_accl_x_s' 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale'

In [16]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.91283,0.91965,0.699342
1,AK,1173,973,0.89656,0.88691,0.623918
2,AP,1305,1180,0.87080,0.86130,0.643929
3,CC,1917,1048,0.88602,0.83931,0.522303
4,MG,821,1107,0.88380,0.89912,0.729489
5,MP,987,1265,0.85837,0.87465,0.719363
6,NM,1107,739,0.89435,0.87014,0.571760
7,RV,1039,681,0.85641,0.81555,0.567264
8,SCh,459,514,0.92500,0.92895,0.691325
9,SM,1261,1067,0.86211,0.85005,0.628571


# VA-Mag-WE (We do not report this in the paper)

In [18]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:

    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])
    
    gdata = data.copy()
    
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(gdata)
    display(vc)
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    
    #VA-Mag-WE
    drop_list = [x for x in X.columns.values if '3d' not in x or ('vel' not in x and 'accl' not in x)]
    X = X.drop(columns = drop_list)
    print('Columns included:', X.columns.values)
    
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])

    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)


    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)

    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)



    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid
    
#     display(results)

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,0.239863,1.698129,1.639468,2.452563,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,7.632676,7.650916,7.199229,12.047121,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,0.195151,0.854034,0.953142,1.188277,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,4.355223,2.669664,2.760062,5.381629,0
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,0.097496,1.194241,1.068156,2.996836,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,3.197070,6.036242,4.899836,14.006587,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.052214,0.666526,0.554291,2.758534,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,2.291594,3.981845,2.569355,11.180447,1
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.277212,3.086626,1.620276,3.760366,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,8.906563,15.135886,8.142713,26.922117,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,0.182422,0.991676,0.706405,2.022844,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,3.496218,3.272292,2.275699,11.566228,0
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,0.133230,1.382107,0.740968,3.661150,-0.134559,0.12

1    956
0    822
Name: Target, dtype: int64

Columns included: ['RWrist_vel_3d_m' 'LWrist_vel_3d_m' 'RElbow_vel_3d_m' 'LElbow_vel_3d_m'
 'RWrist_accl_3d_m' 'LWrist_accl_3d_m' 'RElbow_accl_3d_m'
 'LElbow_accl_3d_m' 'RWrist_vel_3d_s' 'LWrist_vel_3d_s' 'RElbow_vel_3d_s'
 'LElbow_vel_3d_s' 'RWrist_accl_3d_s' 'LWrist_accl_3d_s'
 'RElbow_accl_3d_s' 'LElbow_accl_3d_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 10
Best parameters: {'svc__C': 100, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.8339021099552137
AK


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,2.074682,1.017248,1.856553,1.123659,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,10.267226,5.354898,8.993397,6.269162,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,1.321782,0.714755,0.978453,0.388700,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,4.316787,2.195364,4.123532,2.086301,0
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,1.677389,0.938211,1.530244,1.020210,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,8.903397,4.709948,7.516006,5.893906,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,1.353965,0.643009,1.038240,0.396043,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,4.489218,2.357535,4.469689,1.993289,0
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,1.482856,1.038457,1.370255,1.004904,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,8.184738,4.957916,6.578263,5.447635,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,1.155603,0.707483,0.932994,0.361364,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,4.165420,2.323531,3.945540,1.937088,0
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,0.676299,0.511897,0.804668,0.8

...

0    1261
1    1067
Name: Target, dtype: int64

Columns included: ['RWrist_vel_3d_m' 'LWrist_vel_3d_m' 'RElbow_vel_3d_m' 'LElbow_vel_3d_m'
 'RWrist_accl_3d_m' 'LWrist_accl_3d_m' 'RElbow_accl_3d_m'
 'LElbow_accl_3d_m' 'RWrist_vel_3d_s' 'LWrist_vel_3d_s' 'RElbow_vel_3d_s'
 'LElbow_vel_3d_s' 'RWrist_accl_3d_s' 'LWrist_accl_3d_s'
 'RElbow_accl_3d_s' 'LElbow_accl_3d_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.7898855170210213
SS


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,2.672908,1.087595,3.003405,1.837792,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,6.693822,3.637398,11.719747,6.435866,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,1.212221,0.769910,0.799974,1.100834,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,3.072107,2.505105,4.543724,3.403451,0
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,2.159400,0.667067,6.369831,1.201926,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,5.682507,2.659807,26.248907,6.663064,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,0.720230,0.421421,5.176665,0.540927,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,2.393053,1.242323,13.407714,3.240611,1
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,6.808293,2.144596,4.187057,2.917348,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,7.301063,5.718990,28.685607,7.125793,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,0.601989,0.384581,0.496834,0.357974,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,2.699513,3.241842,29.361424,3.073313,1
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,1.636551,2.825120,1.280379,2.787136,3.

0    1419
1     645
Name: Target, dtype: int64

Columns included: ['RWrist_vel_3d_m' 'LWrist_vel_3d_m' 'RElbow_vel_3d_m' 'LElbow_vel_3d_m'
 'RWrist_accl_3d_m' 'LWrist_accl_3d_m' 'RElbow_accl_3d_m'
 'LElbow_accl_3d_m' 'RWrist_vel_3d_s' 'LWrist_vel_3d_s' 'RElbow_vel_3d_s'
 'LElbow_vel_3d_s' 'RWrist_accl_3d_s' 'LWrist_accl_3d_s'
 'RElbow_accl_3d_s' 'LElbow_accl_3d_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.6374732629365932


In [19]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.82287,0.83389,0.699342
1,AK,1173,973,0.81827,0.79614,0.623918
2,AP,1305,1180,0.79722,0.77661,0.643929
3,CC,1917,1048,0.83304,0.75843,0.522303
4,MG,821,1107,0.80759,0.83539,0.729489
5,MP,987,1265,0.81040,0.83175,0.719363
6,NM,1107,739,0.82287,0.77856,0.571760
7,RV,1039,681,0.78024,0.69900,0.567264
8,SCh,459,514,0.83246,0.84135,0.691325
9,SM,1261,1067,0.81014,0.78987,0.628571


# PVA-L-WE : Only Left Wrist+Elbow (We do not report this in the paper)

In [20]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:
# for singer in ['AG']:
    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])
    
    gdata = data.copy()
    
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(gdata)
    display(vc)
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    
    #PVA-L-WE
    drop_list = [x for x in X.columns.values if '3d' in x or 'RWrist' in x or 'RElbow' in x]
    X = X.drop(columns = drop_list)
    print('Columns included:', X.columns.values)
    
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])

    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)


    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)

    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)



    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid
    
#     display(results)

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,0.239863,1.698129,1.639468,2.452563,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,7.632676,7.650916,7.199229,12.047121,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,0.195151,0.854034,0.953142,1.188277,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,4.355223,2.669664,2.760062,5.381629,0
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,0.097496,1.194241,1.068156,2.996836,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,3.197070,6.036242,4.899836,14.006587,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.052214,0.666526,0.554291,2.758534,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,2.291594,3.981845,2.569355,11.180447,1
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.277212,3.086626,1.620276,3.760366,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,8.906563,15.135886,8.142713,26.922117,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,0.182422,0.991676,0.706405,2.022844,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,3.496218,3.272292,2.275699,11.566228,0
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,0.133230,1.382107,0.740968,3.661150,-0.134559,0.12

1    956
0    822
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'LWrist_x_m' 'LWrist_y_m'
 'LWrist_z_m' 'LWrist_vel_x_m' 'LWrist_vel_y_m' 'LWrist_vel_z_m'
 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'LWrist_x_s'
 'LWrist_y_s' 'LWrist_z_s' 'LWrist_vel_x_s' 'LWrist_vel_y_s'
 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s' 'LElbow_vel_z_s'
 'LWrist_accl_x_s' 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s'
 'LElbow_accl_y_s' 'LElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.9119184498424596
AK


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,2.074682,1.017248,1.856553,1.123659,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,10.267226,5.354898,8.993397,6.269162,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,1.321782,0.714755,0.978453,0.388700,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,4.316787,2.195364,4.123532,2.086301,0
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,1.677389,0.938211,1.530244,1.020210,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,8.903397,4.709948,7.516006,5.893906,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,1.353965,0.643009,1.038240,0.396043,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,4.489218,2.357535,4.469689,1.993289,0
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,1.482856,1.038457,1.370255,1.004904,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,8.184738,4.957916,6.578263,5.447635,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,1.155603,0.707483,0.932994,0.361364,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,4.165420,2.323531,3.945540,1.937088,0
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,0.676299,0.511897,0.804668,0.8

...

0    1261
1    1067
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'LWrist_x_m' 'LWrist_y_m'
 'LWrist_z_m' 'LWrist_vel_x_m' 'LWrist_vel_y_m' 'LWrist_vel_z_m'
 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'LWrist_x_s'
 'LWrist_y_s' 'LWrist_z_s' 'LWrist_vel_x_s' 'LWrist_vel_y_s'
 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s' 'LElbow_vel_z_s'
 'LWrist_accl_x_s' 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s'
 'LElbow_accl_y_s' 'LElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 8
Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.8599139620359159
SS


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,2.672908,1.087595,3.003405,1.837792,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,6.693822,3.637398,11.719747,6.435866,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,1.212221,0.769910,0.799974,1.100834,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,3.072107,2.505105,4.543724,3.403451,0
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,2.159400,0.667067,6.369831,1.201926,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,5.682507,2.659807,26.248907,6.663064,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,0.720230,0.421421,5.176665,0.540927,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,2.393053,1.242323,13.407714,3.240611,1
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,6.808293,2.144596,4.187057,2.917348,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,7.301063,5.718990,28.685607,7.125793,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,0.601989,0.384581,0.496834,0.357974,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,2.699513,3.241842,29.361424,3.073313,1
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,1.636551,2.825120,1.280379,2.787136,3.

0    1419
1     645
Name: Target, dtype: int64

Columns included: ['LElbow_x_m' 'LElbow_y_m' 'LElbow_z_m' 'LWrist_x_m' 'LWrist_y_m'
 'LWrist_z_m' 'LWrist_vel_x_m' 'LWrist_vel_y_m' 'LWrist_vel_z_m'
 'LElbow_vel_x_m' 'LElbow_vel_y_m' 'LElbow_vel_z_m' 'LWrist_accl_x_m'
 'LWrist_accl_y_m' 'LWrist_accl_z_m' 'LElbow_accl_x_m' 'LElbow_accl_y_m'
 'LElbow_accl_z_m' 'LElbow_x_s' 'LElbow_y_s' 'LElbow_z_s' 'LWrist_x_s'
 'LWrist_y_s' 'LWrist_z_s' 'LWrist_vel_x_s' 'LWrist_vel_y_s'
 'LWrist_vel_z_s' 'LElbow_vel_x_s' 'LElbow_vel_y_s' 'LElbow_vel_z_s'
 'LWrist_accl_x_s' 'LWrist_accl_y_s' 'LWrist_accl_z_s' 'LElbow_accl_x_s'
 'LElbow_accl_y_s' 'LElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.7079667874447424


In [21]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.90496,0.91192,0.699342
1,AK,1173,973,0.86348,0.85096,0.623918
2,AP,1305,1180,0.85394,0.84440,0.643929
3,CC,1917,1048,0.85364,0.79797,0.522303
4,MG,821,1107,0.83506,0.85843,0.729489
5,MP,987,1265,0.87171,0.88826,0.719363
6,NM,1107,739,0.86731,0.83700,0.571760
7,RV,1039,681,0.84826,0.80346,0.567264
8,SCh,459,514,0.90235,0.91190,0.691325
9,SM,1261,1067,0.87155,0.85991,0.628571


# PVA-R - Only Right Wrist and Right Elbow (We do not report this in the paper)

In [22]:
all_pred_dict = {}
all_test_dict = {}
grid_objects = {}
summary = []

for singer in ['AG','AK','AP','CC','MG','MP','NM','RV','SCh','SM','SS']:

    print(singer)
    with open(f'./Subseq_Dict_With_Elbow/subseq_dict_{singer}.pkl','rb') as f:
        subseq_dict_new=pickle.load(f)

    length_filter_min = 50
    length_filter_max = 500    

    X_y=dict([(k,v) for k,v in subseq_dict_new.items() \
         if (v.shape[0]>=length_filter_min and v.shape[0] <= length_filter_max)])


    data = []

    for i,key in enumerate(list(X_y.keys())):
        data.append(list(X_y[key].mean().drop(['time','pitch']).values)+ \
                    list(X_y[key].std().drop(['time','pitch']).values) + [key[2]])

    col_list = X_y[key].drop(columns=['time','pitch','filename']).columns.values
    data = pd.DataFrame(data, columns = [x+'_m' for x in col_list] + \
                        [x+'_s' for x in col_list] + \
                        ['Target'])
    
    gdata = data.copy()
    
    verbose = 1

    gdata.dropna(inplace=True)
    gdata['Target'] = gdata['Target'].apply(lambda x: int(x=='st'))
    vc = gdata['Target'].value_counts()
    display(gdata)
    display(vc)
    X = gdata.iloc[:, :-1].reset_index(drop=True)
    
    
    #PVA-R-WE
    drop_list = [x for x in X.columns.values if '3d' in x or 'LWrist' in x or 'LElbow' in x]
    X = X.drop(columns = drop_list)
    print('Columns included:', X.columns.values)
    
    scaler = StandardScaler()
    X = pd.DataFrame(X, columns = X.columns.values)
    y = gdata['Target'].reset_index(drop=True)

    pipeline = Pipeline([( "scaler" , StandardScaler()),
                       ("svc",SVC())])

    params_grid = {'svc__C': [0.001,0.01,0.1,1,10,100,1000],
                   'svc__gamma': ['scale','auto'],
                   'svc__kernel': ['rbf']}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    scoring = {"Accuracy": make_scorer(accuracy_score),
               "f1": make_scorer(f1_score, average='binary', pos_label = 1)}
    grid = GridSearchCV(pipeline, params_grid,verbose=verbose,cv=kf, 
                        scoring = scoring, refit = 'f1')
    grid.fit(X, y)


    sample_indices_by_split = []
    for train_index, test_index in grid.cv.split(X,y):
        sample_indices_by_split.append((train_index, test_index))

    best_model=grid.best_estimator_

    print('Best index:', grid.best_index_)
    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)

    all_test = []
    all_pred = []
    results = []
    for i in range(10):
        train_indices=sample_indices_by_split[i][0]
        test_indices=sample_indices_by_split[i][1]
        X_train = X.iloc[train_indices,:].values
        y_train = y.iloc[train_indices].values
        X_test = X.iloc[test_indices].values
        y_test = y.iloc[test_indices].values
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        results.append([i,round(accuracy_score(y_test, y_pred), 4)
                                    ,round(f1_score(y_test, y_pred, average='binary',pos_label=1),4)])
        all_test.append(y_test)
        all_pred.append(y_pred)



    results = pd.DataFrame(results, columns = ['Split number','Accuracy','F1 Score'])

    all_pred_dict[singer] = all_pred
    all_test_dict[singer] = all_test
    grid_objects[singer] = grid
    
#     display(results)

    summary.append([singer, vc.loc[0],vc.loc[1],
                    results['Accuracy'].mean(), 
                    results['F1 Score'].mean()])

AG


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.354788,-0.291050,0.173531,0.817305,1.021133,1.501446,1.550311,2.130316,0.911514,-0.985366,1.741255,-0.557413,-0.072561,-0.048251,-0.868403,-0.166918,0.707666,-0.695134,1.060117,0.133355,-0.120774,0.211576,-0.476972,-0.651660,0.239863,1.698129,1.639468,2.452563,-0.365269,-0.294922,-0.812808,-0.210166,-0.635700,1.578916,-0.489195,-1.384344,1.481583,0.325125,-0.598778,1.462152,7.632676,7.650916,7.199229,12.047121,0.410789,0.332110,0.293936,0.120198,0.230783,0.319201,0.103405,0.320983,0.311931,0.030874,0.029122,0.358296,0.204230,0.148688,1.581788,0.446678,0.873414,1.283659,1.518920,1.378925,1.444181,0.428971,0.844644,1.418287,0.195151,0.854034,0.953142,1.188277,1.143804,0.904388,8.631008,2.645006,5.469771,5.106388,8.365778,8.025104,6.002712,2.208446,4.624227,5.551636,4.355223,2.669664,2.760062,5.381629,0
1,-0.615711,-1.553891,0.622891,0.688969,1.225628,2.368465,1.538218,1.315594,1.621126,-0.954855,1.773989,-0.422953,-0.006831,-0.012910,0.249150,0.210206,0.372417,0.356620,-1.094454,0.745557,0.145008,0.071123,-0.034882,0.358574,0.097496,1.194241,1.068156,2.996836,-0.134802,-0.002807,-0.556240,-0.423046,1.009915,-1.483476,3.075835,3.010435,-1.471038,-0.316318,0.289686,-1.866748,3.197070,6.036242,4.899836,14.006587,0.666234,0.505801,0.212589,0.043426,0.098773,0.273952,0.075029,0.137306,0.283596,0.016861,0.011940,0.163742,0.084120,0.049049,0.775917,0.220311,0.517090,1.117536,3.164644,1.943748,1.025212,0.221528,0.429657,1.040974,0.052214,0.666526,0.554291,2.758534,0.415886,0.241939,3.866850,1.247910,1.957556,6.605488,12.797326,10.069890,6.001348,1.009852,2.184984,4.607318,2.291594,3.981845,2.569355,11.180447,1
2,0.460858,0.274073,0.237466,0.729641,1.337729,1.386549,1.015612,2.605908,0.707498,-1.116779,1.794838,-0.925318,-0.146994,-0.034805,-1.295014,-0.809700,0.904600,0.112019,0.106926,1.263922,0.022098,0.118143,-0.143867,0.201781,0.277212,3.086626,1.620276,3.760366,0.688413,-0.108087,-2.032817,2.837830,-6.690590,0.614804,4.911500,-9.665036,0.641810,1.285029,-4.110204,0.547565,8.906563,15.135886,8.142713,26.922117,0.585408,0.205647,0.067335,0.138765,0.152768,0.145499,0.329010,0.315681,0.169593,0.073800,0.020801,0.413481,0.262699,0.093852,1.275692,1.355868,2.558224,0.850030,3.668573,1.772376,0.386853,0.818172,1.339045,0.782458,0.182422,0.991676,0.706405,2.022844,2.070194,0.713637,9.104511,9.003527,9.375001,4.407980,26.918587,4.303596,1.904971,4.382808,3.952783,4.275929,3.496218,3.272292,2.275699,11.566228,0
3,2.180690,-0.971371,0.095938,0.907797,1.028449,1.078853,1.331458,1.413417,0.578969,-1.055020,1.771644,-0.621713,-0.023289,0.015422,-0.117711,0.417791,0.249448,0.132387,-0.051041,-1.156359,0.066361,-0.047638,0.077054,0.270461,0.133230,1.382107,0.740968,3.661150,-0.134559,0.12

1    956
0    822
Name: Target, dtype: int64

Columns included: ['RElbow_x_m' 'RElbow_y_m' 'RElbow_z_m' 'RWrist_x_m' 'RWrist_y_m'
 'RWrist_z_m' 'RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'RWrist_accl_x_m'
 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'RElbow_x_s' 'RElbow_y_s' 'RElbow_z_s' 'RWrist_x_s'
 'RWrist_y_s' 'RWrist_z_s' 'RWrist_vel_x_s' 'RWrist_vel_y_s'
 'RWrist_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.922548710704038
AK


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,0.598680,-0.358819,-1.013314,-0.384457,0.152134,-0.571895,0.730232,0.914667,-1.352488,0.263035,-0.549660,-1.827943,-0.055694,0.125737,-1.477219,0.027471,-0.031884,-0.822196,-0.153773,0.075582,-0.447727,-1.068503,-0.452301,-0.250394,2.074682,1.017248,1.856553,1.123659,-0.959995,3.179396,-0.251731,0.049846,0.145124,-0.972123,-0.894613,-0.475037,-0.399623,-0.276586,-2.177416,-0.245958,10.267226,5.354898,8.993397,6.269162,0.087179,0.185155,0.163384,0.279293,0.151562,0.239911,0.014376,0.027105,0.327911,0.081167,0.251264,0.463603,0.449194,1.710336,1.058202,0.061867,0.094405,0.926351,0.423079,0.729395,0.693881,0.655985,1.163732,1.107962,1.321782,0.714755,0.978453,0.388700,4.071325,7.541655,6.347506,0.315492,0.588011,5.684328,2.314721,4.110559,4.531265,4.276867,5.598389,6.633730,4.316787,2.195364,4.123532,2.086301,0
1,0.597181,-0.366134,-0.942230,-0.302876,0.169224,-0.531606,0.729037,0.917299,-1.281191,0.287526,-0.493616,-1.696663,-0.120161,0.121559,-1.062185,0.010670,-0.025570,-0.462600,-0.032698,0.084501,-0.249317,-0.859674,-0.387689,-0.097645,1.677389,0.938211,1.530244,1.020210,-1.037799,1.923784,-1.085616,0.070834,0.102608,-1.041779,-0.478491,0.027535,-1.069750,-0.585414,-1.942599,-0.144527,8.903397,4.709948,7.516006,5.893906,0.088215,0.166728,0.191824,0.284032,0.136511,0.224418,0.012772,0.024189,0.316979,0.084098,0.240917,0.467668,0.410898,1.491904,1.196093,0.065404,0.085300,1.035177,0.435002,0.643171,0.728393,0.688051,1.022223,1.005728,1.353965,0.643009,1.038240,0.396043,3.561617,6.946609,5.743739,0.285368,0.528057,5.139444,2.280776,3.705645,4.314472,3.772040,4.900182,5.867066,4.489218,2.357535,4.469689,1.993289,0
2,0.599991,-0.372843,-0.895387,-0.228953,0.176775,-0.454257,0.732547,0.915849,-1.207462,0.304084,-0.498170,-1.595340,0.044952,0.301654,-0.950756,0.013842,-0.001041,-0.657787,-0.017616,0.136890,-0.332317,-0.535077,-0.300464,-0.345487,1.482856,1.038457,1.370255,1.004904,-1.145354,1.350326,0.553383,-0.020101,0.100999,0.532032,-0.562156,-0.435246,0.308398,-0.597134,-0.885005,0.532719,8.184738,4.957916,6.578263,5.447635,0.079110,0.146073,0.184187,0.258048,0.120422,0.242358,0.016196,0.026221,0.310108,0.081125,0.209339,0.432297,0.438028,1.258381,1.250512,0.069846,0.106223,1.064263,0.425837,0.591965,0.694946,0.739411,0.888905,0.959781,1.155603,0.707483,0.932994,0.361364,2.980418,5.880948,6.140318,0.336554,0.576807,5.417040,2.224787,3.249981,4.176268,3.129323,4.467003,5.277317,4.165420,2.323531,3.945540,1.937088,0
3,0.481876,-0.620479,-0.355912,-0.237339,0.404825,0.066016,0.718289,0.910667,-0.538390,0.119008,-0.654150,-0.948814,0.187045,-0.304531,-0.010239,0.031106,-0.022872,-0.142531,0.162773,0.300094,-0.692059,0.431440,-0.404719,-0.259170,0.676299,0.511897,0.804668,0.8

...

0    1261
1    1067
Name: Target, dtype: int64

Columns included: ['RElbow_x_m' 'RElbow_y_m' 'RElbow_z_m' 'RWrist_x_m' 'RWrist_y_m'
 'RWrist_z_m' 'RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'RWrist_accl_x_m'
 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'RElbow_x_s' 'RElbow_y_s' 'RElbow_z_s' 'RWrist_x_s'
 'RWrist_y_s' 'RWrist_z_s' 'RWrist_vel_x_s' 'RWrist_vel_y_s'
 'RWrist_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 10
Best parameters: {'svc__C': 100, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.8360164495964628
SS


,LElbow_x_m,LElbow_y_m,LElbow_z_m,RElbow_x_m,RElbow_y_m,RElbow_z_m,LWrist_x_m,LWrist_y_m,LWrist_z_m,RWrist_x_m,RWrist_y_m,RWrist_z_m,RWrist_vel_x_m,RWrist_vel_y_m,RWrist_vel_z_m,LWrist_vel_x_m,LWrist_vel_y_m,LWrist_vel_z_m,LElbow_vel_x_m,LElbow_vel_y_m,LElbow_vel_z_m,RElbow_vel_x_m,RElbow_vel_y_m,RElbow_vel_z_m,RWrist_vel_3d_m,LWrist_vel_3d_m,RElbow_vel_3d_m,LElbow_vel_3d_m,RWrist_accl_x_m,RWrist_accl_y_m,RWrist_accl_z_m,LWrist_accl_x_m,LWrist_accl_y_m,LWrist_accl_z_m,LElbow_accl_x_m,LElbow_accl_y_m,LElbow_accl_z_m,RElbow_accl_x_m,RElbow_accl_y_m,RElbow_accl_z_m,RWrist_accl_3d_m,LWrist_accl_3d_m,RElbow_accl_3d_m,LElbow_accl_3d_m,LElbow_x_s,LElbow_y_s,LElbow_z_s,RElbow_x_s,RElbow_y_s,RElbow_z_s,LWrist_x_s,LWrist_y_s,LWrist_z_s,RWrist_x_s,RWrist_y_s,RWrist_z_s,RWrist_vel_x_s,RWrist_vel_y_s,RWrist_vel_z_s,LWrist_vel_x_s,LWrist_vel_y_s,LWrist_vel_z_s,LElbow_vel_x_s,LElbow_vel_y_s,LElbow_vel_z_s,RElbow_vel_x_s,RElbow_vel_y_s,RElbow_vel_z_s,RWrist_vel_3d_s,LWrist_vel_3d_s,RElbow_vel_3d_s,LElbow_vel_3d_s,RWrist_accl_x_s,RWrist_accl_y_s,RWrist_accl_z_s,LWrist_accl_x_s,LWrist_accl_y_s,LWrist_accl_z_s,LElbow_accl_x_s,LElbow_accl_y_s,LElbow_accl_z_s,RElbow_accl_x_s,RElbow_accl_y_s,RElbow_accl_z_s,RWrist_accl_3d_s,LWrist_accl_3d_s,RElbow_accl_3d_s,LElbow_accl_3d_s,Target
0,-1.008607,0.726924,0.284048,-0.733272,-1.078791,-0.085289,-0.027196,1.148404,-0.383745,-0.554662,0.100878,-0.772714,1.255110,1.486877,-0.417992,-0.086879,0.076571,-0.939678,-0.096096,-0.508112,-1.537304,1.135705,0.599649,-1.237122,2.672908,1.087595,3.003405,1.837792,-1.167308,0.136109,0.747042,-0.515169,0.198751,0.414387,0.099051,0.145724,0.642276,-1.234556,1.001667,0.028892,6.693822,3.637398,11.719747,6.435866,0.112923,0.330378,0.819645,0.593440,0.482051,0.692653,0.058495,0.050523,0.557355,0.740311,0.821159,0.333732,1.089655,0.846839,0.800507,0.329274,0.193423,0.857051,0.443652,0.648218,1.159563,1.222036,1.974595,1.054317,1.212221,0.769910,0.799974,1.100834,5.479889,3.361940,3.338596,1.176588,0.644701,4.157020,2.391039,2.788070,6.263741,6.412096,9.323726,5.288121,3.072107,2.505105,4.543724,3.403451,0
1,-0.938090,1.125555,1.709723,-1.783144,0.780412,1.033072,-0.165185,1.205083,0.580048,-2.622942,-1.358823,-0.145745,1.756515,0.679352,-0.077811,0.177607,-0.090292,0.066754,0.173064,-0.215335,-0.259302,-0.905956,-5.355003,-0.076547,2.159400,0.667067,6.369831,1.201926,0.650773,0.998841,-0.532308,0.003247,-0.037705,-1.488260,-1.311508,-0.750952,-2.270587,2.816992,1.086377,-2.021728,5.682507,2.659807,26.248907,6.663064,0.168148,0.096301,0.182595,0.584916,2.409724,0.123442,0.080123,0.036373,0.109826,0.565883,0.268810,0.060578,0.811626,0.509151,0.344629,0.184566,0.049162,0.737765,0.708542,0.313804,1.000409,2.226444,5.676216,0.870964,0.720230,0.421421,5.176665,0.540927,5.206890,2.741371,1.366073,1.006602,0.282109,2.310267,3.835420,1.838743,5.439154,9.580938,27.443855,3.818984,2.393053,1.242323,13.407714,3.240611,1
2,-0.963112,1.013807,-0.338327,1.163198,-1.696511,0.297405,-0.364664,1.238780,0.250691,-0.784025,-0.565359,0.214264,-6.806029,-0.062493,1.504933,0.009747,0.026207,2.138951,-0.834428,-0.559958,2.656504,-1.698986,-2.353754,2.688659,6.808293,2.144596,4.187057,2.917348,-2.736533,-1.093501,-2.145296,-0.013886,0.667641,-1.888030,-3.153538,2.978698,4.603656,-4.691307,23.886570,-2.751520,7.301063,5.718990,28.685607,7.125793,0.127859,0.097456,0.337485,0.228543,0.367659,0.358854,0.012804,0.014111,0.279962,0.732750,0.027360,0.210595,0.603113,0.105003,0.339271,0.098986,0.098824,0.390050,0.493013,0.421056,0.399411,0.799666,1.027855,0.677553,0.601989,0.384581,0.496834,0.357974,4.037128,1.519704,5.421099,0.607537,0.338242,6.262351,3.116277,0.708582,3.191149,1.529340,32.045978,7.704946,2.699513,3.241842,29.361424,3.073313,1
3,-0.043546,-0.093324,-0.364586,0.692009,-0.644612,-0.902429,0.408035,0.035068,-0.848655,0.091234,0.232859,-0.844978,0.422589,0.459306,0.099229,-2.133781,1.588298,0.294820,-1.232961,2.119356,0.028525,-0.612990,0.289162,0.517008,1.636551,2.825120,1.280379,2.787136,3.

0    1419
1     645
Name: Target, dtype: int64

Columns included: ['RElbow_x_m' 'RElbow_y_m' 'RElbow_z_m' 'RWrist_x_m' 'RWrist_y_m'
 'RWrist_z_m' 'RWrist_vel_x_m' 'RWrist_vel_y_m' 'RWrist_vel_z_m'
 'RElbow_vel_x_m' 'RElbow_vel_y_m' 'RElbow_vel_z_m' 'RWrist_accl_x_m'
 'RWrist_accl_y_m' 'RWrist_accl_z_m' 'RElbow_accl_x_m' 'RElbow_accl_y_m'
 'RElbow_accl_z_m' 'RElbow_x_s' 'RElbow_y_s' 'RElbow_z_s' 'RWrist_x_s'
 'RWrist_y_s' 'RWrist_z_s' 'RWrist_vel_x_s' 'RWrist_vel_y_s'
 'RWrist_vel_z_s' 'RElbow_vel_x_s' 'RElbow_vel_y_s' 'RElbow_vel_z_s'
 'RWrist_accl_x_s' 'RWrist_accl_y_s' 'RWrist_accl_z_s' 'RElbow_accl_x_s'
 'RElbow_accl_y_s' 'RElbow_accl_z_s']
Fitting 10 folds for each of 14 candidates, totalling 140 fits
Best index: 6
Best parameters: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best score: 0.7557919977889194


In [23]:
svm_results = pd.DataFrame(summary,columns=['Singer','Non-stable','Stable','Accuracy','F1'])
svm_results['Chance F1'] = svm_results.apply(lambda df: chance_F1(df['Non-stable'],df['Stable']), axis=1)
svm_results

,Singer,Non-stable,Stable,Accuracy,F1,Chance F1
0,AG,822,956,0.91453,0.92256,0.699342
1,AK,1173,973,0.91427,0.90653,0.623918
2,AP,1305,1180,0.89857,0.89135,0.643929
3,CC,1917,1048,0.89207,0.84721,0.522303
4,MG,821,1107,0.87085,0.88731,0.729489
5,MP,987,1265,0.85747,0.87364,0.719363
6,NM,1107,739,0.91384,0.89350,0.571760
7,RV,1039,681,0.87035,0.82904,0.567264
8,SCh,459,514,0.90239,0.90723,0.691325
9,SM,1261,1067,0.84922,0.83603,0.628571
